In [ ]:
%matplotlib inline
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, date2num
import matplotlib.ticker as ticker
import geopandas
from shapely.geometry import shape
from PIL import Image
import glob
from IPython.display import HTML

# Load data from nytimes
# Has data for each county in each state
df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv', 
                 parse_dates=['date'])



In [ ]:
# Print the data
df.head()

In [ ]:
# Select South Carolina
sc = df[df.state == "South Carolina"]
# Group by county for each date
sc = sc.pivot(index='date', columns='county', values='cases')
sc = sc.fillna(0.0)

print(sc.memory_usage(deep=True))
sc.tail(5)

In [ ]:
plot = sc.plot(figsize=(12,8), linewidth=5)
plot.legend(ncol=2, bbox_to_anchor=(1, 1), loc='upper left')
plt.show()

In [ ]:
# Show plotting inside of pandas
plot = sc.Richland.plot(figsize=(12,8), linewidth=5, legend=True)
plot = sc.Lexington.plot(linewidth=5, legend=True)

In [ ]:
# Show plotting inside of pandas (log this time)
plot = sc.Richland.plot(figsize=(12,8), linewidth=5, legend=True)
plot = sc.Lexington.plot(linewidth=5, legend=True)
plot.set_yscale('log')

In [ ]:
# Show how to plot dates with matplotlib plot_date

fig, ax = plt.subplots(figsize=(16,9))

days_to_average = 5

dates = date2num(sc.index)
ax.plot_date(dates, sc.Richland.rolling(days_to_average).mean(), fmt='-', linewidth=5)
ax.plot_date(dates, sc.Lexington.rolling(days_to_average).mean(), fmt='-', linewidth=5)
date_form = DateFormatter("%b-%d")
ax.xaxis.set_major_formatter(date_form)
ax.set_yscale('log')
plt.show()

In [ ]:
# Copy data
new_per_day = sc.copy(deep=True)

# Get difference per day instead of total cases
for county in sc.columns:
    new_per_day[county] = new_per_day[county].diff()
new_per_day = new_per_day.fillna(0.0)
new_per_day.tail(5) 


In [ ]:
# Plot difference
# Looks jumpy....
plot = new_per_day.Richland.plot(figsize=(12,8), linewidth=5, legend=True)
#plot.set_yscale('log')

In [ ]:
# Plot again using matplotib
fig, ax = plt.subplots(figsize=(16,9))

rolling_days = 7

dates = date2num(sc.index)
# This time we do rolling mean to take some of the jittering out
ax.plot_date(dates, new_per_day.Richland.rolling(rolling_days).mean(), fmt='-', linewidth=5)
date_form = DateFormatter("%b-%d")
ax.xaxis.set_major_formatter(date_form)
plt.show()

In [ ]:
# Get geometry data
geodata = geopandas.read_file("https://raw.githubusercontent.com/deldersveld/topojson/master/countries/us-states/SC-45-south-carolina-counties.json")

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
geodata.plot(color="white", edgecolor='black', ax=ax)
max_count = np.max(sc.max(axis=0))

geodata["color"] = np.zeros_like(geodata.NAME)

for c in sc.iteritems():
    geodata.loc[geodata.NAME == c[0], 'color'] = c[1][-1]/max_count

geodata.plot(column='color', ax=ax, cmap='PuBu')
ax.set_title("Current Cases in SC")
ax.set_yticklabels([])
ax.set_xticklabels([])
ax.get_yaxis().set_visible(False)
ax.get_xaxis().set_visible(False)
plt.show()
plt.close()

In [ ]:
# Get max from data and max of all the counties
max_count = np.max(sc.T.max(axis=0))
total = sc.T.sum()

# Count is used to save images
count = 0

os.mkdir("sc_gif") if not os.path.isdir("sc_gif") else None

for date, row in sc.T.iteritems():
    fig, ax = plt.subplots(figsize=(16,9))
    geodata.plot(color='white',ax=ax, edgecolor='k')
    for c in row.iteritems():
        color = c[1]/max_count
        geodata.loc[geodata.NAME == c[0], 'color'] = color
    
    geodata.plot(column='color', ax=ax, cmap='PuBu')

    ax.set_title(f"{date.month_name()} {date.day}\nTotal Cases: {int(total[count])}")
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.get_yaxis().set_visible(False)
    ax.get_xaxis().set_visible(False)
    count += 1
    fig.savefig(f"sc_gif/{count:03}.png")
    plt.close()

In [ ]:
# Create the frames
frames = []
imgs = glob.glob("sc_gif/*.png")
imgs = np.sort(imgs)

for i in imgs:
    new_frame = Image.open(i)
    frames.append(new_frame)

# Pause on the last frame for longer
for i in range(0,20):
    frames.append(Image.open(imgs[-1]))

# Save into a GIF file that loops forever
frames[0].save('cases_in_sc.gif', format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=600, loop=0)


In [ ]:
# Show gif in jupyter
HTML('<img src="cases_in_sc.gif">')